In [62]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix, classification_report
import timeit
import matplotlib.pyplot as plt

In [63]:
!pip install nltk

In [64]:
import nltk

In [65]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /usr/share/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /usr/share/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /usr/share/nltk_data...
[nltk_data]    |   Pack

True

In [66]:
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
ps=PorterStemmer()

In [67]:
fake=pd.read_csv('../input/newsdata/test.csv')

In [68]:
true=pd.read_csv('../input/newsdata/train.csv', encoding = "ISO-8859-1")

In [69]:
true['label']=1
fake['label']=0

In [70]:
fake.shape,true.shape

((150, 6), (350, 6))

In [71]:
df=pd.concat([fake,true])

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 349
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    500 non-null    object
 1   text     500 non-null    object
 2   subject  500 non-null    object
 3   date     500 non-null    object
 4   is_fake  500 non-null    int64 
 5   label    500 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 27.3+ KB


In [73]:
df[df.duplicated()]

Empty DataFrame
Columns: [title, text, subject, date, is_fake, label]
Index: []

In [74]:
df=df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 349
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    500 non-null    object
 1   text     500 non-null    object
 2   subject  500 non-null    object
 3   date     500 non-null    object
 4   is_fake  500 non-null    int64 
 5   label    500 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 27.3+ KB


In [75]:
df=df.reset_index(drop=True)

In [76]:
df.head()

title  \
0  REPORTER TO CLINTON: ‘So you still blame other...   
1  U.S. Patent and Trademark Office head Michelle...   
2  POLICE IN GERMANY BEGIN RAIDS On Homes Of Face...   
3  BUSTED: [VIDEO] MAN ATTEMPTS TO TAPE “GOTCHA” ...   
4  Erdogan says U.S. stance stalls Turkish ratifi...   

                                                text       subject  \
0  British TV journalist Matt Frei asked Hillary ...      politics   
1  (Reuters) - U.S. Patent and Trademark Office D...  politicsNews   
2  If Facebook has aligned themselves with German...      politics   
3  The Michael Brown and Eric Garner cases have o...     left-news   
4  HAMBURG (Reuters) - The U.S. decision to pull ...  politicsNews   

            date  is_fake  label  
0   Oct 13, 2017        1      0  
1  June 6, 2017         0      0  
2   Jul 15, 2016        1      0  
3    Apr 5, 2015        1      0  
4  July 8, 2017         0      0

In [77]:
df=df.drop(columns=['date'],axis=1)

In [78]:
df.subject.value_counts()

politicsNews       116
News               109
worldnews           98
politics            87
left-news           55
Government News     19
Middle-east         10
US_News              6
Name: subject, dtype: int64

In [79]:
import re
corpus=[]
for i in range(len(df)):
  review=re.sub('[^a-zA-Z]',' ',df.title[i])
  review=review.lower()
  review=review.split()
  review=[ps.stem(i) for i in review if i not in set(stopwords.words('english'))]
  review=' '.join(review)
  corpus.append(review)

corpus  

['report clinton still blame other loss video',
 'u patent trademark offic head michel lee resign',
 'polic germani begin raid home facebook user post hate speech refuge',
 'bust video man attempt tape gotcha video cop abus power get unexpect challeng local news',
 'erdogan say u stanc stall turkish ratif pari climat deal',
 'billionair green donor launch u millenni voter drive',
 'watch trump assum black report set meet black lawmak',
 'hous intel chair trump russia fake stori evid anyth video',
 'tillerson get first brief state depart spokesman',
 'donald trump want jail american refus spi neighbor',
 'mma fighter jake shield embarrass coward mask violent beat trump support video berkeley watch man get beat mob polic help person jump help',
 'gender equal justic unw american mother father abroad',
 'anarchist threaten disrupt trump inaugur polic say readi',
 'conserv fox news houston host fire say think obama race war facebook',
 'japan pm abe rate regain percent amid north korea sec

In [80]:
len(corpus)

500

In [81]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Dense,LSTM,Embedding
from tensorflow.keras.models import Sequential

In [82]:
def bench_mmm(startn,maxn,step,loops):
    count=0
    
    #Preallocate results lists
    avg_gflops = int((1+(maxn-startn)/step))*[0] 
    peak_gflops = int((1+(maxn-startn)/step))*[0]
    raw_times = [int(loops)*[0] for i in range(int(1+(maxn-startn)/step))]
    all_gflops = [int(loops)*[0] for i in  range(int(1+(maxn-startn)/step))]
    mat_size=int((1+(maxn-startn)/step))*[0] 

    for n in range(startn,maxn+step,step):
        setup_string = "from pylab import rand,dot;n=%d;a=rand(n,n);b=rand(n,n)" % n
        time_list = timeit.repeat("a.dot(b)", setup=setup_string, repeat=loops,number=1)
        raw_times[count] = time_list
        total_time = sum(time_list)
        avg_time = total_time / loops
        peak_time = min(time_list)
        num_ops = 2*n**3-n**2
        avg_gflops[count] = (num_ops/avg_time)/10**9
        peak_gflops[count] = (num_ops/peak_time)/10**9
        all_gflops[count] = [(num_ops/time)/10**9 for time in raw_times[count]]        
        mat_size[count] = n
        count=count+1
    
    plt.plot(mat_size,avg_gflops,'*-',label="Average over %d runs" %loops)
    plt.plot(mat_size,peak_gflops,'*-',label="Peak")
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.xlabel('Matrix Size');
    plt.ylabel('GFlop/s');
    plt.show()
    
    return(max(peak_gflops),raw_times,all_gflops)

In [83]:
vocab_size=10000
one_hot_repr=[one_hot(word,vocab_size) for word in corpus]
one_hot_repr[0]

[1075, 7621, 1218, 6101, 6477, 410, 3915]

In [84]:
sent_length=20
pad_doc=pad_sequences(one_hot_repr,padding='pre',maxlen=sent_length)
pad_doc[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 1075, 7621, 1218, 6101, 6477,  410, 3915], dtype=int32)

In [85]:
emb_vec_fea=100
model=Sequential()
model.add(Embedding(vocab_size,emb_vec_fea,input_length=sent_length))
model.add(LSTM(200))
model.add(Dense(1,activation='sigmoid'))


In [86]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics='accuracy')

In [87]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 100)           1000000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 200)               240800    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 201       
Total params: 1,241,001
Trainable params: 1,241,001
Non-trainable params: 0
_________________________________________________________________


In [88]:
X=pad_doc
y=df.label

In [89]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [90]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=15,batch_size=100)

Epoch 1/15
4/4 [==============================] - 3s 280ms/step - loss: 0.6801 - accuracy: 0.6900 - val_loss: 0.6618 - val_accuracy: 0.6700
Epoch 2/15
4/4 [==============================] - 0s 83ms/step - loss: 0.6250 - accuracy: 0.7075 - val_loss: 0.6743 - val_accuracy: 0.6700
Epoch 3/15
4/4 [==============================] - 0s 82ms/step - loss: 0.6013 - accuracy: 0.7075 - val_loss: 0.6459 - val_accuracy: 0.6700
Epoch 4/15
4/4 [==============================] - 0s 83ms/step - loss: 0.5841 - accuracy: 0.7075 - val_loss: 0.6529 - val_accuracy: 0.6700
Epoch 5/15
4/4 [==============================] - 0s 81ms/step - loss: 0.5637 - accuracy: 0.7075 - val_loss: 0.6677 - val_accuracy: 0.6700
Epoch 6/15
4/4 [==============================] - 0s 80ms/step - loss: 0.5286 - accuracy: 0.7075 - val_loss: 0.6464 - val_accuracy: 0.6700
Epoch 7/15
4/4 [==============================] - 0s 81ms/step - loss: 0.4866 - accuracy: 0.7200 - val_loss: 0.6688 - val_accuracy: 0.6600
Epoch 8/15
4/4 [==========

In [91]:
y_pred=model.predict(X_test)
y_pred=y_pred>0.5
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.31      0.15      0.20        33
           1       0.67      0.84      0.74        67

    accuracy                           0.61       100
   macro avg       0.49      0.49      0.47       100
weighted avg       0.55      0.61      0.56       100

